In [14]:
# Housekeeping, imports
import pandas as pd
import numpy as np
import os
from IPython.display import Markdown

In [15]:
# Housekeeping, imports
df = pd.read_excel('zib_translaties.xlsx', sheet_name='translatie')
zibinfo = pd.read_excel('zib_versions.xlsx', sheet_name='table_info_zibs')
zibinfo = zibinfo.fillna('')
zibinfo

,zib,wiki_URL_2020,wiki_URL_2017,OID_zib,sub_header_1,sub_header_2
0,Ademhaling,https://zibs.nl/wiki/Ademhaling-v3.2(2020NL),https://zibs.nl/wiki/Ademhaling-v3.1(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.12.5,,
1,Adresgegevens,https://zibs.nl/wiki/Adresgegevens-v1.1(2020NL),https://zibs.nl/wiki/Adresgegevens-v1.0(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.20.5,,
2,AlcoholGebruik,https://zibs.nl/wiki/AlcoholGebruik-v3.2(2020NL),https://zibs.nl/wiki/AlcoholGebruik-v3.1(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.7.3,,
3,Alert,https://zibs.nl/wiki/Alert-v4.1(2020NL),https://zibs.nl/wiki/Alert-v3.2(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.8.1,zie ZIB-1769 de leveranciers hebben gevraagd w...,zie ook https://www.rivm.nl/antibioticaresiste...
4,AlgemeneMeting,deprecated in 2020,https://zibs.nl/wiki/AlgemeneMeting-v3.0(2017NL),2.16.840.1.113883.2.4.3.11.60.40.3.13.3,De zib AlgemeneMeting komt te vervallen. Voor ...,
...,...,...,...,...,...,...
110,ZorgAfspraak,https://zibs.nl/wiki/ZorgAfspraak-v1.0(2020NL),nieuw in 2020,2.16.840.1.113883.2.4.3.11.60.40.3.16.2,,
111,ZorgEpisode,https://zibs.nl/wiki/ZorgEpisode-v1.0(2020NL),nieuw in 2020,2.16.840.1.113883.2.4.3.11.60.40.3.16.6,,
112,ZorgTeam,https://zibs.nl/wiki/ZorgTeam-v1.0(2020NL),nieuw in 2020,2.16.840.1.113883.2.4.3.11.60.40.3.17.3,,
113,JuridischeSituatie,https://zibs.nl/wiki/JuridischeSituatie-v2.0(2...,nieuw in 2020,2.16.840.1.113883.2.4.3.11.60.40.3.7.17,,


In [16]:
# Replace nulls with emtpy string
df = df.fillna('')
# Replace characters which trouble XML
cols = df.columns
# newcols = [col.replace('->', '_').replace('-', '_').replace('>', '_') for col in cols]
# df.columns = newcols
# cols = df.columns
cols

Index(['O_Sort1', 'O_Sort2', 'O_Sort3', 'O_status', 'O_DCM_ID', 'H_ZibName',
       'T_ConceptID_2017', 'T_ConceptName_2017', 'T_Codelists_2017',
       'T_Change', 'T_ConceptID_2020', 'T_ConceptName_2020',
       'T_Codelists_2020', 'T_Bits', 'T_Omschrijving', 'T_TypeChange',
       'T_Impact_heen', 'T_TRANSLATIE_spec_heen', 'T_Impact_terug',
       'T_TRANSLATIE_spec_terug'],
      dtype='object')

In [17]:
# Make a list of header and table column names
hcols = []
tcols = []
for col in cols:
    if col[0:2] == "H_":
        hcols.append(col[2:])
    elif col[0:2] == "T_":
        tcols.append(col[2:])
    else:
        pass
newcols = [col[2:] for col in cols]
df.columns = newcols
newcols

['Sort1',
 'Sort2',
 'Sort3',
 'status',
 'DCM_ID',
 'ZibName',
 'ConceptID_2017',
 'ConceptName_2017',
 'Codelists_2017',
 'Change',
 'ConceptID_2020',
 'ConceptName_2020',
 'Codelists_2020',
 'Bits',
 'Omschrijving',
 'TypeChange',
 'Impact_heen',
 'TRANSLATIE_spec_heen',
 'Impact_terug',
 'TRANSLATIE_spec_terug']

In [18]:
# Make a list of zibs.
zibs = df['ZibName'].dropna().unique()
# zibs

In [19]:
# Create folders
if not(os.path.exists('mapping')):
   os.mkdir(mapping)
for format in ['csv', 'md', 'json', 'xml', 'excel']:
    if not(os.path.exists(os.path.join('mapping', format))):
       os.mkdir(os.path.join('mapping', format))

In [30]:
# Markdown. Make a header.
smallcols = ['ConceptID_2017', 'ConceptName_2017', 'Change', 'TypeChange', 'Impact_heen', 'TRANSLATIE_spec_heen', 'Impact_terug', 'TRANSLATIE_spec_terug',  'Omschrijving']
header = "# {zibname}\n## File formats\n\nThe translation specs are available as: \n[CSV](../csv/{zibname}.csv) [JSON](../json/{zibname}.json) [XML](../xml/{zibname}.xml)\n\n"
subheader = ""
shcols = ['ConceptID_2017', 'ConceptName_2017', 'Change', 'TypeChange', 'Omschrijving']
for zib in zibs:
    tdf = df[df['ZibName'] == zib]
    tdf = tdf.fillna('')
    subheader = "## Zib " + zib + "\n\nDCM::Id:\t" + str(tdf.iloc[0]['DCM_ID']) + "\n\nDCM::Name:\t" + str(tdf.iloc[0]['ZibName']) + "\n\n"
    url = zibinfo[zibinfo['zib'] == zib]['wiki_URL_2017'].item()
    subheader = subheader + "[" + url.split('/')[-1] + "](" + url +")\n\n"
    url = zibinfo[zibinfo['zib'] == zib]['wiki_URL_2020'].item()
    subheader = subheader + "[" + url.split('/')[-1] + "](" + url +")\n\n"
    subheader = subheader + "\n\n" + zibinfo[zibinfo['zib'] == zib]['sub_header_1'].item()
    subheader = subheader + "\n\n" + zibinfo[zibinfo['zib'] == zib]['sub_header_2'].item()
    htable = tdf[(tdf['ConceptID_2017'].str.contains("DCM::"))][shcols]
    if not(htable.empty):
        mdhtable = '## Zib-level changes\n\n' + htable.to_markdown(index=False)
    else:
        mdhtable = ''
    mapping = tdf[~tdf['ConceptID_2017'].str.contains("DCM::")][tcols].to_markdown(index=False)
    changes = tdf[~tdf['ConceptID_2017'].str.contains("DCM::") & ~tdf['Change'].str.contains("groen")][smallcols].to_markdown(index=False)
    with open(os.path.join('mapping', 'md', zib + '.md'), 'w', encoding='utf8') as file:
        content = header.format(zibname = zib) + '\n\n' + subheader + '\n\n' + mdhtable + '\n\n## Changes\n\n' + changes  + '\n\n## Mapping\n\n' + mapping + '\n\n'
        file.write(content)
Markdown(content)

# AnatomischeLocatie
## File formats

The translation specs are available as: 
[CSV](../csv/AnatomischeLocatie.csv) [JSON](../json/AnatomischeLocatie.json) [XML](../xml/AnatomischeLocatie.xml)



## Zib AnatomischeLocatie

DCM::Id:	20.7

DCM::Name:	AnatomischeLocatie

[nieuw in 2020](nieuw in 2020)

[AnatomischeLocatie-v1.0(2020NL)](https://zibs.nl/wiki/AnatomischeLocatie-v1.0(2020NL))









## Changes

| ConceptID_2017             | ConceptName_2017   | Change   | TypeChange   | Impact_heen   | TRANSLATIE_spec_heen   | Impact_terug   | TRANSLATIE_spec_terug   | Omschrijving   |
|:---------------------------|:-------------------|:---------|:-------------|:--------------|:-----------------------|:---------------|:------------------------|:---------------|
| **zib toegevoegd in 2020** |                    |          |              |               |                        |                |                         |                |

## Mapping

| ConceptID_2017             | ConceptName_2017   | Codelists_2017   | Change   | ConceptID_2020             | ConceptName_2020   | Codelists_2020   | Bits   | Omschrijving   | TypeChange   | Impact_heen   | TRANSLATIE_spec_heen   | Impact_terug   | TRANSLATIE_spec_terug   |
|:---------------------------|:-------------------|:-----------------|:---------|:---------------------------|:-------------------|:-----------------|:-------|:---------------|:-------------|:--------------|:-----------------------|:---------------|:------------------------|
| **zib toegevoegd in 2020** |                    |                  |          | **zib toegevoegd in 2020** |                    |                  |        |                |              |               |                        |                |                         |



In [8]:
"https://zibs.nl/wiki/Ademhaling-v3.1(2017NL)".split('/')[-1]

'Ademhaling-v3.1(2017NL)'

In [9]:
for zib in zibs:
    tdf = df[df['ZibName'] == zib][hcols + tcols]
    tdf.to_csv(os.path.join('mapping', 'csv', zib + '.csv'), index=False)

In [10]:
for zib in zibs:
    tdf = df[df['ZibName'] == zib][hcols + tcols]
    tdf.to_xml(os.path.join('mapping', 'xml', zib + '.xml'), index=False)

In [11]:
for zib in zibs:
    tdf = df[df['ZibName'] == zib][hcols + tcols]
    tdf.to_json(os.path.join('mapping', 'json', zib + '.json'), orient='records')

In [12]:
for zib in zibs:
    tdf = df[df['ZibName'] == zib][hcols + tcols]
    tdf.to_excel(os.path.join('mapping', 'excel', zib + '.xlsx'), index=False)

In [13]:
mdindex = "# Index of available zib translations\n\n"
for zib in sorted(zibs):
    mdindex = mdindex + "* [ " + zib + "](mapping/md/" + zib + ".md)\n"
with open('index.md', 'w', encoding='utf8') as file:
    file.write(mdindex)